In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text
import numpy as np
import tarfile as tar
import jsonlines
from tqdm.notebook import tqdm

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [2]:
# Google saves the day, yet again.
model = hub.load('https://tfhub.dev/google/universal-sentence-encoder-multilingual-qa/3')

In [3]:
question_encoder = model.signatures['question_encoder']
response_encoder = model.signatures['response_encoder']

In [4]:
res = []

with jsonlines.open('valid.jsonl') as f:
    for line in tqdm(f.iter(), position=0, total=10000):
        q = tf.constant([line['question']])
        r = tf.constant(line['candidates'])
        q_emb = question_encoder(q)
        r_emb = response_encoder(input=r, context=r)
        res.append(np.argmax(np.inner(q_emb['outputs'], r_emb['outputs'])))

  0%|          | 0/10000 [00:00<?, ?it/s]

In [5]:
with open('res.csv', 'w') as f:
    f.write('Id,Category\n')
    for i in range(len(res)):
        f.write(f'{i},{res[i]}\n')